In [ ]:
import pandas as pd
from megatron import saving as m
from simple_salesforce import bulk
from connect import td_query as td
from connect import sf_query as sf
#from connect import sf_login as sfl
from datetime import date


In [ ]:
### This is the BUMBLEBEE output in a dataframe

# connect and query Teradata
bumblebee = td('SELECT DISTINCT account_id,current_owner_id,employee_sf_id FROM sandbox.bumblebee_output where employee_sf_id IS NOT NULL')
bumblebee['account_id'] = bumblebee['account_id'].str.strip()


In [ ]:
### This is the current account ownership in SF

# define variables
account_ids = bumblebee['account_id'].tolist()
owner_ids = []

# get data from SOQL query and add to dictionary
for account_id in account_ids:
    soql_output = sf('SELECT Account_ID_18__c,OwnerId FROM Account where Account_ID_18__c',account_id)
    owner_id = soql_output['records'][0]['OwnerId']
    owner_ids.append(owner_id)
    
data = {
    'account_id': account_ids,
    'owner_id': owner_ids
}

current_owner_table = pd.DataFrame(data)


In [ ]:
### The filtered table is ready for upload to SF

# This joins the current_owner_table and the bumblebee table
df = bumblebee.join(current_owner_table.set_index('account_id'), on='account_id')

# this filters the table on current_owner_id in Teradata compared to owner_id in SF, and also on the intended employee_sf_id to the current owner_id in SF
filtered_table = df.drop(df[(df['current_owner_id'] != df['owner_id']) & (df['owner_id'] == df['employee_sf_id'])].index)

# this drops unnecesary columns
filtered_table.drop('current_owner_id', axis=1, inplace=True)
filtered_table.drop('owner_id', axis=1, inplace=True)

# columns are renamed to fit Salesforce names
filtered_table.columns = ['Id','OwnerId']


In [ ]:
### The data is converted into an ordered dictionary, which is a requirement for SF bulk upload
ready_for_upload = []

for row in filtered_table.itertuples():
    d = row._asdict()
    del d['Index']
    ready_for_upload.append(d)


In [ ]:
### updates are made in Salesforce
from simple_salesforce import Salesforce as sf
    
login_sf = sf(username = 'svc_rao@groupon.com', password = 'Password-1Je8IR8frn0gkbS68Eq4OHZmM', security_token = '')

login_sf.bulk.Account.update(ready_for_upload,batch_size=10000,use_serial=True)


In [ ]:
## upload data to Teradata for metadata reporting
load_date = str(date.today())
filtered_table['load_date'] = load_date

m.df_to_sql(filtered_table,table_name='bumblebee_processed_changes', adapter='teradata')

print("Success")